# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/likejun/become-data-engineer/P0B_DataModeling


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[:5])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Harmonia', 'Logged In', 'Ryan', 'M', '0', 'Smith', '655.77751', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Sehr kosmisch', '200', '1.54224E+12', '26'], ['The Prodigy', 'Logged In', 'Ryan', 'M', '1', 'Smith', '260.07465', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'The Big Gundown', '200', '1.54224E+12', '26'], ['Train', 'Logged In', 'Ryan', 'M', '2', 'Smith', '205.45261', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Marry Me', '200', '1.54224E+12', '26'], ['', 'Logged In', 'Wyatt', 'M', '0', 'Scott', '', 'free', 'Eureka-Arcata-Fortuna, CA', 'GET', 'Home', '1.54087E+12', '563', '', '200', '1.54225E+12', '9'], ['', 'Logged In', 'Austin', 'M', '0', 'Rosales', '', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'GET', 'Home', '1.54106E+12', '521', '', '200', '1.54225E+12', '12']]


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [10]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['192.168.22.2', '192.168.22.3'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [11]:
# Create a Keyspace 
session.execute("""
CREATE KEYSPACE IF NOT EXISTS dend
WITH REPLICATION = 
{ 'class': 'SimpleStrategy', 'replication_factor': 3}
""")

#### Set Keyspace

In [12]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('dend')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [13]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
def executeQuery(session, query):
    try:
        session.execute(query)
    except Exception as e:
        print(e)

query = 'DROP TABLE IF EXISTS music_table1'
executeQuery(session, query)

# session_id determines the partition while item_in_session controls the 
# clustering column(primary key in partition and the order)
query = 'CREATE TABLE IF NOT EXISTS music_table1 '
query += '(artist TEXT, song TEXT, length FLOAT, session_id INT, item_in_session INT, PRIMARY KEY(session_id, item_in_session))'
executeQuery(session, query)

In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_table1(artist, song, length, session_id, item_in_session)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
## Add in the SELECT statement to verify the data was entered into the table
query = 'SELECT artist, song, length FROM music_table1 WHERE session_id = 338 AND item_in_session = 4'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


From the above, we can see that the song "Music Matters (Mark Knight Dub)" with the artist Faithless lasts for 495.31 seconds with the session ID equals to 338 and  item ID in session equals to 4.

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
## Query 2: Give me only the following: name of artist, 
## song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = 'DROP TABLE IF EXISTS music_table2'
executeQuery(session, query)

query = 'CREATE TABLE IF NOT EXISTS music_table2'
query += '(artist TEXT, song TEXT, item_in_session INT, first_name TEXT, last_name TEXT, user_id INT, session_id INT, PRIMARY KEY((user_id, session_id), item_in_session))'
executeQuery(session, query)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_table2(artist, song, item_in_session, first_name, last_name, user_id, session_id)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))

query = 'SELECT artist, item_in_session, song, first_name, last_name FROM music_table2 WHERE session_id = 182 AND user_id = 10'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print('{}-{}-{}-{}-{}'.format(row.artist, row.item_in_session, row.song, row.first_name, row.last_name))

Down To The Bone-0-Keep On Keepin' On-Sylvie-Cruz
Three Drives-1-Greece 2000-Sylvie-Cruz
Sebastien Tellier-2-Kilometer-Sylvie-Cruz
Lonnie Gordon-3-Catch You Baby (Steve Pitron & Max Sanna Radio Edit)-Sylvie-Cruz


With session ID 182 and user ID 10, there are four records, all sorted by item ID in session.

In [17]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = 'DROP TABLE IF EXISTS music_table3'
executeQuery(session, query)

query = 'CREATE TABLE IF NOT EXISTS music_table3'
query += '(song TEXT, first_name TEXT, last_name TEXT, PRIMARY KEY(song))'
executeQuery(session, query)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_table3(song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1], line[4]))

query = "SELECT first_name, last_name FROM music_table3 WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch


Sara Johnson listened to the song "All Hands Against His Own".

### Drop the tables before closing out the sessions

In [18]:
executeQuery(session, 'DROP TABLE music_table1')
executeQuery(session, 'DROP TABLE music_table2')
executeQuery(session, 'DROP TABLE music_table3')

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()